In [1]:
# 04_grid_random_search (n)
# 1. Zaimplementuj GridSearchCV oraz RandomizedSearchCV dla datasetu wine. Uwzględnij poniższe parametry:
# - estymator: LogisticRegression(solver="liblinear")
#  - parametr C:
#     - min 1
#     - max 10 000
#     - liczba wystąpień 1 000
#  - regularyzacja l1 oraz l2
# 2. Zaimplementuj GridSearchCV (jeden na wszystkie modele) w celu znalezienia najlepszego algorytmu oraz hyperparametrów dla datasetu z pkt.1:
#  - wykorzystaj estymatory:
#     - RandomForestClassifier
#     - KNeighborsClassifier
#     - LogisticRegression
# 3. Porównaj wyniki korzystając z  hyperopt-sklearn.

# Rozwiązanie prześlij jako printscreen (jpg, pdf).
# Nazwa pliku (bez polskich znaków):

# Nazwisko_Imie_04_stacj.jpg